## data
### for grabbing / processing (census) data

In [ ]:
from gerrytools.data import *
import geopandas as gpd
import pandas as pd
import us

_Note: Sometimes, when calling any of the below functions, you may get an error code that looks like this:_
```
ValueError: Unexpected response (URL: ...): Sorry, the system is currently undergoing maintenance or is busy.  Please try again later. 
```
_This is due to a Census API issue and can't be fixed on our end. Usually, running the function again works like a charm!_

### census
Uses the US Census Bureau's API to retrieve 2020 Decennnial Census PL 94-171 data at the stated geometry level. The five tables are
 * P1: Race
 * P2: Hispanic or Latino, and Not Hispanic or Latino by Race
 * P3: Race for the Population 18 Years and Over (Race by VAP)
 * P4: Hispanic or Latino, and Not Hispanic or Latino by Race for the Population 18 Years and Over
 * P5: Group Quarters Population by Major Group Quarters Type

In [ ]:
us.states.MA

In [ ]:
%%time
# this should take < 5s
df = census20(
    us.states.MA, 
    table="P3", # Table from which we retrieve data, defaults to "P1"
    columns={}, # mapping Census column names from the table to human-readable names, if desired
    geometry="tract", # data granularity, one of "block" (default), "block group", or "tract"
)

df[["GEOID20", "VAP20", "WHITEVAP20", "BLACKVAP20", "ASIANVAP20", "OTHVAP20"]].head()

In [ ]:
# The `variables()` function produces the default mapping that `census()` uses 
# to map Census column-names to human-readable ones
mapping = variables("P3")
mapping

variables("P3")

In [ ]:
# Census column P3_001N is total Voting Age Population
mapping['P3_001N']

### acs5
Uses the US Census Bureau's API to retrieve 5-year population estimates from the American Community Survey (ACS) for the provided state, geometry level, and year.

In [ ]:
%%time 
# this should take < 1 min
acs5_df = acs5(
    us.states.MA,
    geometry="block group", # data granularity, either "tract" (default) or "block group"
    year=2019, # Year for which data is retrieved. Defaults to 2019, i.e. 2015-19 ACS 5-year
)
acs5_df[["BLOCKGROUP10", "TOTPOP19", "WHITE19", "BLACK19", "ASIAN19", "OTH19"]].head()

### cvap
Uses the US Census Bureau's API to retrieve the 2019 5-year CVAP (Citizen Voting Age Population) data for the provided state at the specified geometry. Please note that the geometries are from the **2010 Census**.

In [ ]:
%%time
# this should take < 15s
cvap_df = cvap(
    us.states.MA,
    geometry="block group", # data granularity, either "tract" (default) or "block group"
    year=2019
)
cvap_df.head()

### estimating cvap
This function wraps the above `cvap()` and `acs5()` functions to help users pull forward CVAP estimates from 2019 (on 2010 geometries) to estimates for 2020 (on 2020 geometries). To use this, one must supply a base geodataframe with the 2020 geometries on which they want CVAP estimates. Additionally, users must specify the demographic groups whose CVAP statistics are to be estimated. For each group, users specify a triple $(X, Y, Z)$ where $X$ is the old CVAP column for that group, $Y$ is the old VAP column for that group, and $Z$ is the new VAP column for that group, which must be an existing column on `base`.  Then, the estimated new CVAP for that group will be constructed by multiplying $X / Y \cdot Z$ for each new geometry.

In [ ]:
%%time
base = gpd.read_file("data/al_bg/") # Load AL 2020 block-group shapefile
acs5_cvap19 = acs5(us.states.AL, year=2019) # Get CVAP19 estimates from ACS
cvap_cvap19 = cvap(us.states.AL, year=2019) # Get CVAP19 estimates from ACS Special Tabulation

#### Tips for picking $X$, $Y$, and $Z$:

$X$ should be any CVAP column returned by either `acs5()` or `cvap()`, so anything from the following list:

In [ ]:
print([col for col in pd.concat([acs5_cvap19, cvap_cvap19]) if "CVAP" in col])

Note that the `acs5()` method returns things like `BCVAP19` or `HCVAP19` (Black-alone CVAP and Hispanic CVAP, respectively) while the `cvap()` method returns things like `NHBCVAP19` (Non-Hispanic Black-alone CVAP). There are also columns like `NHBCWVAP19`, which refer to all Non-Hispanic citizens of voting age who self-identified as Black and White. However, since your choice of $Y$ is restricted to single-race or ethnicity columns (see below), we recommend only estimating CVAP for single-race or ethnicity columns, like `BCVAP19`, `HCVAP19`, or `NHBCVAP19`).

In [ ]:
print([col for col in pd.concat([acs5_cvap19, cvap_cvap19]) if "VAP" in col and "CVAP" not in col])

Lastly, one should choose $Z$ to match one's choice for $Y$ (say, `BVAP20` to match `BVAP19`). However, in some cases it is reasonable to choose a $Z$ that is a close but imperfect match. For example, setting $(X, Y, Z) = $ `(BCVAP19, BVAP19, APBVAP20)` (where $Z = $ `APBVAP` refers to all people of voting age who selected Black alone or in combination with other Census-defined races) would allow one to estimate the 2020 CVAP population of people who selected Black alone or in combination with other races.

One final note: there are some instances in which, due to small Census reporting discrepancies, the `acs5()` and the `cvap()` methods disagree on CVAP19 estimates (this might happen for total `CVAP19` or `HCVAP19`, for example). In these cases we default to the `acs5()` numbers.

In [ ]:
acs5_cvap19.columns

In [ ]:
estimates = estimatecvap2010(
    base,
    us.states.AL,
    groups=[ # (Old CVAP, Old VAP, new VAP)
        ("WHITECVAP19", "WHITEVAP19", "WVAP20"),
        ("BLACKCVAP19", "BLACKVAP19", "BVAP20"),
    ],
    ceiling=1, # see below
    zfill=0.1, # see below
    geometry10="tract"
)

The `ceiling` parameter marks when we will cap the CVAP / VAP ratio to 1. Set to 1, this means that if there is ever more CVAP19 in a geometry than VAP19, we will "cap" the CVAP20 estimate to 100\% of the VAP20. The `zfill` parameter tells us what to do when there is 0 CVAP19 in a geometry. Set to 0.1, this will estimate that 10\% of the VAP20 is CVAP.

We can see that our estimate for Black-alone Voting Age Population in Alabama in 2020 is 970,120, down slightly from 970,239 in 2019.

In [ ]:
print(estimates.columns)

In [ ]:
print(f"AL BLACKCVAP20: {estimates.BLACKCVAP20_EST.sum()}")
print(f"AL BLACKCVAP19: {acs5_cvap19.BLACKCVAP19.sum()}")

We can also make estimates of Black VAP in Alabama among `APBVAP` — Alabamians who identified as Black alone or in combination with other races. This bumps up the estimate to around 1,007,363.

In [ ]:
estimates = estimatecvap2010(
    base,
    us.states.AL,
    groups=[
        # Changing the new VAP column from BVAP20 -> APBVAP20
        ("BLACKCVAP19", "BLACKVAP19", "APBVAP20"),
    ],
    ceiling=1,
    zfill=0.1,
    geometry10="tract"
)

In [ ]:
print(f"AL APBCVAP20 estimate: {estimates.BLACKCVAP20_EST.sum()}")